In [1]:
import pandas as pd # 데이터프레임 사용을 위해
from math import log # IDF 계산을 위해

In [2]:
docs = [
  '먹고 싶은 사과',
  '먹고 싶은 바나나',
  '길고 노란 바나나 바나나',
  '저는 과일이 좋아요'
] 
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()

In [3]:
[w for doc in docs for w in doc.split()]

['먹고',
 '싶은',
 '사과',
 '먹고',
 '싶은',
 '바나나',
 '길고',
 '노란',
 '바나나',
 '바나나',
 '저는',
 '과일이',
 '좋아요']

In [4]:
N = len(docs) # 총 문서의 수

def tf(t, d):
    return d.count(t)

def idf(t):
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df + 1))

def tfidf(t, d):
    return tf(t,d)* idf(t)

In [5]:
result = []
for i in range(N): # 각 문서에 대해서 아래 명령을 수행 4번
    result.append([])
    d = docs[i]
    for j in range(len(vocab)): #9번
        t = vocab[j]        
        result[-1].append(tf(t, d))

tf_ = pd.DataFrame(result, columns = vocab)
tf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


In [6]:
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index = vocab, columns = ["IDF"])
idf_

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


In [7]:
result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]

        result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


# 사이킷런을 이용한 DTM과 TF-IDF 실습

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]
vector = CountVectorizer()
print(vector.fit_transform(corpus).toarray()) # 코퍼스로부터 각 단어의 빈도 수를 기록한다.
print(vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.
#dtm (document term matrix 생성)

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


DTM이 완성되었습니다. DTM에서 각 단어의 인덱스가 어떻게 부여되었는지를 확인하기 위해, 인덱스를 확인해보았습니다. 첫번째 열의 경우에는 0의 인덱스를 가진 do입니다. do는 세번째 문서에만 등장했기 때문에, 세번째 행에서만 1의 값을 가집니다. 두번째 열의 경우에는 1의 인덱스를 가진 know입니다. know는 첫번째 문서에만 등장했기 때문에 첫번째 행에서만 1의 값을 가집니다.

사이킷런은 TF-IDF를 자동 계산해주는 TfidfVectorizer를 제공합니다. 향후 실습을 하다가 혼란이 생기지 않도록 언급하자면, 사이킷런의 TF-IDF는 위에서 배웠던 보편적인 TF-IDF 식에서 좀 더 조정된 다른 식을 사용합니다. 하지만 크게 다른 식은 아니며(IDF의 로그항의 분자에 1을 더해주며, 로그항에 1을 더해주고, TF-IDF에 L2 정규화라는 방법으로 값을 조정하는 등의 차이), 여전히 TF-IDF가 가진 의도를 그대로 갖고 있으므로 사이킷런의 TF-IDF를 그대로 사용하셔도 좋습니다.

In [102]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]
tfidfv = TfidfVectorizer().fit(corpus) #min_df/analyzer='char' option 사용해볼수 있다.
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


In [13]:
tfidfv = TfidfVectorizer(min_df=2).fit(corpus) #min_df/analyzer='char' option 사용해볼수 있다.
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[1.]
 [1.]
 [0.]]
{'you': 0}


In [12]:
tfidfv = TfidfVectorizer(analyzer='char').fit(corpus) #min_df/analyzer='char' option 사용해볼수 있다.
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0.54522682 0.14041576 0.         0.14041576 0.         0.10904536
  0.14041576 0.10904536 0.3692597  0.43618145 0.18462985 0.
  0.14041576 0.21809073 0.18462985 0.28083152 0.28083152]
 [0.50040087 0.         0.         0.32217861 0.         0.50040087
  0.32217861 0.25020043 0.         0.25020043 0.         0.
  0.         0.25020043 0.         0.         0.32217861]
 [0.54546812 0.17559738 0.4617789  0.         0.4617789  0.13636703
  0.         0.13636703 0.         0.27273406 0.         0.23088945
  0.17559738 0.13636703 0.         0.17559738 0.        ]]
{'y': 16, 'o': 9, 'u': 13, ' ': 0, 'k': 6, 'n': 8, 'w': 15, 'i': 5, 'a': 1, 't': 12, 'r': 10, 'l': 7, 'v': 14, 'e': 3, 'h': 4, 's': 11, 'd': 2}


In [15]:
tfidfv = TfidfVectorizer(ngram_range=(1,2)).fit(corpus) #min_df/analyzer='char' option 사용해볼수 있다.
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0.         0.34142622 0.34142622 0.         0.         0.34142622
  0.         0.         0.34142622 0.34142622 0.         0.
  0.25966344 0.34142622 0.34142622 0.34142622]
 [0.         0.         0.         0.62276601 0.62276601 0.
  0.         0.         0.         0.         0.         0.
  0.4736296  0.         0.         0.        ]
 [0.4472136  0.         0.         0.         0.         0.
  0.4472136  0.4472136  0.         0.         0.4472136  0.4472136
  0.         0.         0.         0.        ]]
{'you': 12, 'know': 1, 'want': 8, 'your': 14, 'love': 5, 'you know': 13, 'know want': 2, 'want your': 9, 'your love': 15, 'like': 3, 'like you': 4, 'what': 10, 'should': 6, 'do': 0, 'what should': 11, 'should do': 7}


이렇게 옵션을 주어서 의도하는 모델을 만들어 내야한다.

### 유클리디안 거리

In [31]:
# euclidean
docs = [doc1,doc2,doc3]

for doc in docs:
    dist = np.sqrt(np.square(np.abs(docInput - doc)).sum())
    print(f'입력 문서와 {doc} 사이 거리 : ',dist)

입력 문서와 [2 3 0 1] 사이 거리 :  2.23606797749979
입력 문서와 [1 2 3 1] 사이 거리 :  3.1622776601683795
입력 문서와 [2 1 2 2] 사이 거리 :  2.449489742783178


### 코사인 유사도

In [47]:
from numpy import dot
from numpy.linalg import norm
import numpy as np
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))

In [51]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(a,b):
    return dot(a,b) / (norm(a)*norm(b))

In [49]:
doc1=np.array([2,3,0,1])
doc2=np.array([1,2,3,1])
doc3=np.array([2,1,2,2])
docInput=np.array((1,1,0,1))

In [52]:
# cosine similarity
docs = [doc1,doc2,doc3]

for doc in docs:
    similarity = cos_sim(docInput, doc)
    print(f'{doc}와 {docInput}의 코사인유사도:', similarity)

[2 3 0 1]와 [1 1 0 1]의 코사인유사도: 0.9258200997725515
[1 2 3 1]와 [1 1 0 1]의 코사인유사도: 0.5962847939999439
[2 1 2 2]와 [1 1 0 1]의 코사인유사도: 0.8006407690254358


### 자카드 유사도

자카드 유사도 알고리즘
교(집합): 두 집합에서 공통으로 가지고 있는 원소 집합
합(집합): 두 집합에서 가지고 있는 원소 집합
합집합에서 교집합의 비율을 구하자. => 두 문서의 유사도

A와 B 두개의 집합이 있다고 합시다. 이때 교집합은 두 개의 집합에서 공통으로 가지고 있는 원소들의 집합을 말합니다. 즉, 합집합에서 교집합의 비율을 구한다면 두 집합 A와 B의 유사도를 구할 수 있다는 것이 자카드 유사도(jaccard similarity)의 아이디어입니다. 자카드 유사도는 0과 1사이의 값을 가지며, 만약 두 집합이 동일하다면 1의 값을 가지고, 두 집합의 공통 원소가 없다면 0의 값을 갖습니다. 자카드 유사도를 구하는 함수를 J
라고 하였을 때, 자카드 유사도 함수 J
는 아래와 같습니다.  

J(doc1, doc2) = (doc1 교 doc2) / (doc1 합 doc2)

$$J(A,B)\; = \,{|A∩B|\over|A∪B|} = {{|A∩B|}\over{|A|+|B|−|A∩B|}}$$

In [37]:
# 다음과 같은 두 개의 문서가 있습니다.
# 두 문서 모두에서 등장한 단어는 apple과 banana 2개.
doc1 = "apple banana everyone like likey watch card holder"
doc2 = "apple banana coupon passport love you"

# 토큰화를 수행합니다.
tokenized_doc1 = doc1.split()
tokenized_doc2 = doc2.split()

# 토큰화 결과 출력
print(tokenized_doc1)
print(tokenized_doc2)

['apple', 'banana', 'everyone', 'like', 'likey', 'watch', 'card', 'holder']
['apple', 'banana', 'coupon', 'passport', 'love', 'you']


In [38]:
union = set(tokenized_doc1).union(set(tokenized_doc2))
print(union)

{'like', 'banana', 'apple', 'coupon', 'everyone', 'love', 'card', 'you', 'holder', 'likey', 'passport', 'watch'}


In [39]:
intersection = set(tokenized_doc1).intersection(set(tokenized_doc2))
print(intersection)

{'apple', 'banana'}


In [40]:
print(len(intersection)/len(union)) # 2를 12로 나눔.

0.16666666666666666


# 영화 추천시스템 구현

### Metadata
- 데이터를 설명해주거나 꾸며주는 데이터를 메타데이터라고 한다.
- 영화에대한 이해를 돕기위한 데이터
- 유트브영상이 데이터라면, 메다데이터는 영상의 설명이다.

In [78]:
data=pd.read_csv("movies_metadata.csv", low_memory=False)
#low_memory : 데이터를 읽을때, 데이터의 컬럼에대해 타입이 무엇이다 예측을한다. 그 작업을 하지 않기위해 하는 옵션설정이다.
#대용량의 텍스트 데이터를 읽을때 활용하면 좋다.

In [79]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

### 공분산, 분산
0 <= 피어슨 상관계수 (x,y) = Covariance(x,y) / (std(x)*std(y)) <= 1  
Covariance(x,y) = (Xi-Xmean) * (Yi-Ymean)    
variance(x)= sum((Xi - Xmean)**2) / n  
std(x) = sqrt(variance(x))   

In [80]:
data=data.head(20000)

In [81]:
data.shape

(20000, 24)

In [82]:
data['overview']
#전처리 생략

0        Led by Woody, Andy's toys live happily in his ...
1        When siblings Judy and Peter discover an encha...
2        A family wedding reignites the ancient feud be...
3        Cheated on, mistreated and stepped on, the wom...
4        Just when George Banks has recovered from his ...
                               ...                        
19995    Dissidents in a French colony attack a police ...
19996    A young mother Nina and her son Enzo find them...
19997    An in-depth analysis of the relationship betwe...
19998    Follows the life and work of animator Lotte Re...
19999    An in-depth look at the genesis, production, a...
Name: overview, Length: 20000, dtype: object

In [83]:
data.overview.isnull().sum()

135

In [84]:
data['overview'] = data['overview'].fillna('')

In [85]:
data.overview.isnull().sum()

0

In [114]:
#tfidf
tfidfMat=TfidfVectorizer(stop_words='english')
tfidfMat=tfidfMat.fit_transform(data['overview']) #여기서는 data['overview']가 코퍼스임
tfidfMat.shape

(20000, 47487)

___
연습문제

In [116]:
print(tfidfMat[0].toarray())
print(np.max(tfidfMat[0].toarray()[0])) #특정 단어의 tfidf값
tfidfMat[0].toarray().shape
# np.max(tfidfMat[0].toarray()[0])) 에서나온 0.503(tfidf값)에 해당하는
# 단어가 무엇인지? 인덱스는 몇번인덱스인지? 찾기

[[0. 0. 0. ... 0. 0. 0.]]
0.5036643709879097


(1, 47487)

In [138]:
idx = tfidfMat[0].toarray().argmax() #최대 tidif값에 해당되는 단어의 index

vocab = TfidfVectorizer(stop_words = 'english').fit(data['overview']).vocabulary_

[key for key, value in vocab.items() if value == idx][0]
#최대 tfidf값의 인덱스와 일치할경우 출력

'buzz'

In [139]:
# 퀴즈 2 , 0.5036... 에 해당되는 단어 출력

word_index_dict = TfidfVectorizer(stop_words='english').fit(data['overview']).vocabulary_
index_word_dict = sorted(word_index_dict.items(),key=lambda x:x[1])

idx = np.argmax(tfidfMat[0].toarray()[0])
print(f'해당 단어 : ',index_word_dict[idx][0])

해당 단어 :  buzz


___

In [140]:
from sklearn.metrics.pairwise import linear_kernel

In [141]:
cos_sim = linear_kernel(tfidfMat, tfidfMat)

In [142]:
cos_sim.shape

(20000, 20000)

In [144]:
cos_sim[0].shape #자기자신에대한 모든 영화와의 코사인 유사도

(20000,)

In [147]:
#최대값에 해당하는 영화제목 추출
#테이블의 용도는 영화의 타이틀을 입력하면 인덱스를 리턴하기 위함
indices = pd.Series(data.index, index=data['title']).drop_duplicates()
print(indices.head())

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64


In [148]:
def get_recommendations(title, cos_sim=cos_sim):
    # 선택한 영화의 타이틀로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 영화를 가지고 연산할 수 있습니다.
    idx = indices[title]

    # 모든 영화에 대해서 해당 영화와의 유사도를 구합니다.
    sim_scores = list(enumerate(cos_sim[idx]))

    # 유사도에 따라 영화들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아옵니다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴합니다.
    return data['title'].iloc[movie_indices]

In [149]:
get_recommendations('The Dark Knight Rises')

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object

___
퀴즈

In [175]:
index=indices['The Dark Knight Rises']

scores= list(enumerate(cos_sim[index]))

sorted_scores=sorted(scores, key=lambda x:x[1], reverse=True)

sorted_movie_idx= [i[0] for i in sorted_scores]

data['title'].iloc[sorted_movie_idx][1:21]

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
4363                                Criminal Law
6042                                       Q & A
11753                                  Slow Burn
1491                              Batman & Robin
13838                  The File on Thelma Jordon
1353                    Night Falls on Manhattan
12289                                   Ricochet
3267                                         JFK
18139                           Batman and Robin
9977        The Hitchhiker's Guide to the Galaxy
Name: title, dtype: 

단순히 단어로만 영화의 유사도를 찾고, 유의어 동의어 처리도 되어있지않고

그래서 연관성이 떨어진 영화가 나온것.

In [172]:
title_in = 'Jumanji'


top20_idx = pd.Series(cos_sim[indices[title_in]]).sort_values(ascending=False)[1:21].index
top20_idx_sort = top20_idx.sort_values()

print(f'{title_in} 와 가장 유사한 줄거리의 영화 TOP 20 : ')
print(data.title[top20_idx])

Jumanji 와 가장 유사한 줄거리의 영화 TOP 20 : 
6166                       Brainscan
8801                         Quintet
17223                 The Dark Angel
9503                       Word Wars
13601    The Mindscape of Alan Moore
16843                         DeVour
8079                         Masques
6055                Poolhall Junkies
19726                 Wreck-It Ralph
2486                        eXistenZ
10892                     Stay Alive
13711                     Rhinoceros
1506              The Innocent Sleep
9107                         Nirvana
19345                       The Wave
10703                  Grandma's Boy
7749              The Last of Sheila
14166                          Gamer
12726           Battlefield Baseball
15512                Le Pont du Nord
Name: title, dtype: object
